# Colab setting

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd ./drive/Othercomputers/MacBook/Earth/module/dd_earthquake/book

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/MacBook/Earth/module/dd_earthquake/book


In [2]:
%ls

bench_mark.ipynb  colab_lgbt_multigrid.ipynb  learn_categorical_plots.ipynb
catboost_info/    edm_data.ipynb              Light_GBM.ipynb


Install lgbm with gpu option

ref: https://an-engineer-note.com/?p=624

In [3]:
!pip uninstall --yes lightgbm && pip install --install-option=--gpu lightgbm

Found existing installation: lightgbm 3.3.5
Uninstalling lightgbm-3.3.5:
  Successfully uninstalled lightgbm-3.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached lightgbm-3.3.5.tar.gz (1.5 MB)
  Preparing metadata (setup.py) ... done
Skipping wheel build for lightgbm, due to binaries being disabled for it.
  Running setup.py install for lightgbm ... done


# lgbm

In [4]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('../')

In [5]:
import numpy as np
import pandas as pd
import warnings
import pprint

import lightgbm as lgb
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from src import common

## grid search

In [6]:
train_values, train_labels = common.file.read_data('train')

In [7]:
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,0
669485,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
602512,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0


In [8]:
features_list = ['geo_level_1_id', 
                 'geo_level_2_id', 
                 'geo_level_3_id',
                 'age', 
                 'area_percentage', 
                 'height_percentage', 
                 'foundation_type', 
                 'roof_type', 
                 'ground_floor_type', 
                 'other_floor_type', 
                 'position', 
                 'has_superstructure_mud_mortar_stone', 
                 'has_superstructure_cement_mortar_brick', 
                 'has_superstructure_timber', 
                 'count_families', 
                 'has_secondary_use'
                ]

In [9]:
train_values, train_labels = common.lgbm_preprocessing((train_values, train_labels), mode='training', features_list=features_list)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(train_values, train_labels,
                                                        test_size=0.1, random_state=19, stratify=train_labels)

In [11]:
skf = StratifiedKFold(n_splits=5,
                      shuffle=True,
                      random_state=0)

In [ ]:
# initialize model and parameter grid
model = lgb.LGBMClassifier(objective='multi_class', num_class=3, 
                           learning_rate=0.15, 
                           force_row_wise=True, 
                           num_boost_round=200, 
                           device='gpu', 
                           # valid_sets=valids
                           )

param_grid = {
    "max_depth": [10, 25, 50, 75],
    # "learning_rate" : [0.001,0.01,0.05,0.1],
    # "num_leaves": [100,300,900,1200],
    "n_estimators": [100,200,500], 
    # 'reg_alpha': list(map(lambda x: x*0.1, range(1, 5))), 
    # 'reg_lambda': [0.1, 0.2],
             }

# grid search
grid_result = GridSearchCV(estimator = model,
                           param_grid = param_grid,
                           scoring = 'f1_micro', 
                           cv = skf,
                           return_train_score = True,
                           n_jobs = -1)

grid_result.fit(x_train, np.reshape(y_train, -1))

In [22]:
pprint.pprint(grid_result.best_estimator_)

LGBMClassifier(device='gpu', force_row_wise=True, learning_rate=0.15,
               num_boost_round=100, num_class=3, objective='multi_class',
               reg_lambda=0.1, silent=False)
